資料3737筆

In [1]:
import pandas as pd
import json
import sys
import os
import gensim
import numpy as np
import glob
from gensim.models.doc2vec import Doc2Vec

C:\Users\Big data\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#load json檔案成dataframe,看一下內容
with open('E:/news_project/bgLabel/angle/opinion_printed_all_v2.json', 'r',encoding='UTF-8') as f:
    data = json.load(f)
    df1 = pd.DataFrame.from_dict(data, orient='columns')

df1 = df1.drop(["author"], axis=1)
df1 = df1.drop(["head"], axis=1)
df1 = df1.drop(["date_"], axis=1)
df1 = df1.drop(["type"], axis=1)
df1 = df1.drop(["source"], axis=1)

In [3]:
#丟掉不要的欄位，合併成一個datafram
df_all = pd.DataFrame(columns=['content','tag', 'title'])
for files in glob.glob('E:/news_project/bgLabel/old/*'):
    with open(files,"r", encoding="utf-8") as json_data:
        data = json.load(json_data)
        df = pd.DataFrame.from_dict(data, orient='columns')
    
        df = df.drop(["infor"], axis=1)
        df = df.drop(["date_"], axis=1)
        df = df.drop(["type"], axis=1)
        df = df.drop(["source"], axis=1)

        df_all=pd.concat([df_all,df],axis=0,ignore_index=True )
# df_all=pd.concat([df_all,df1],axis=0,ignore_index=True )


In [122]:
print(df_all.columns)
df=df_all
df.head()

Index(['content', 'tag', 'title', 'd2v_Vec'], dtype='object')


,content,tag,title,d2v_Vec
0,民進 黨 的 主席 選戰 ， 儼 已成 為 「 保皇派 」 和 「 反 保皇派 」 之爭 。...,0,女皇的新衣,"[-1.206782, -1.3442309, -5.4025793, -1.1229587..."
1,葉俊榮 在 宣布 教育部 將發台 大校 長 聘書給 管中 閔 之前 ， 據 說 曾 致電 立...,0,葉部長「勇敢」的代價,"[-0.4745511, -5.0516253, -3.6623557, 0.4304008..."
2,拗 了 又 拗 ， 教育部 長 葉俊榮終 於 宣布 將發台 大校 長 聘書給 管中 閔 ， ...,0,三百天才找回理性,"[-3.8533144, -3.2074935, -0.8450376, 0.3748564..."
3,蘇貞昌將 接閣 揆 已 經傳 了 一陣子 。 儘 管 其 幕僚 否認 此 說 ， 但 多位 ...,0,聽說蘇貞昌接閣揆,"[-2.204379, -1.9464985, -1.8394303, 2.3343253,..."
4,柯文 哲雙 城論壇 再 提 「 兩岸 一家 親 」 ， 又 吹 皺政壇 春水 。 對照 他 ...,0,柯文哲的如意算盤,"[-1.9381931, 1.8546289, -2.243623, 0.55695444,..."


In [5]:
def split_news(news):
    return " ".join(jieba.cut(news))

def process_news(df):
    df['content'] = df ['content'].apply(split_news)
    df['title'] = df['title'].str.replace('\r','')
    df['title'] = df['title'].str.replace('\n','')
    return df

In [6]:
import jieba
df = process_news(df)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.819 seconds.
Prefix dict has been built succesfully.


In [7]:
#取出content列為一個list
b=list(df['content'])

In [8]:
TaggededDocument=gensim.models.doc2vec.TaggedDocument

In [9]:
x_train=[]
for i, text in enumerate(b):
    word_list = text.split(' ')
    l = len(word_list)
    word_list[l-1]=word_list[l-1].strip()
    document=TaggededDocument(word_list,tags=[i])
    x_train.append(document)
#print(x_train)
c = x_train

In [10]:
model=Doc2Vec(x_train, min_count=1, window=3, vector_size=100, negative=5,workers=4)
model.train(x_train,total_examples=model.corpus_count,epochs=10)

In [11]:
model_dm=model

In [113]:
#保存doc2vec模型
import pickle 
with open('ModelSave/doc2vec_Label_P.pickle','wb')as f:
    pickle.dump(model_dm,f)

In [123]:
strl=df.iloc[78,0]
test_text=strl.split(' ')
#得到向量array命名為inferred_vector
inferred_vector=model_dm.infer_vector(doc_words=test_text,alpha=0.025,steps=500)

sims=model_dm.docvecs.most_similar([inferred_vector],topn=3)
sims

C:\Users\Big data\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(78, 0.7087225914001465),
 (3293, 0.47007647156715393),
 (3275, 0.4531461298465729)]

In [124]:
#找到與特定文章strl相似的文章

for count, sim in sims:
    print(count,sim)
    sentence = c[count]
    words = ''
    
    for word in sentence[0]:
        words = words + word +' '
    print(words,sim,len(sentence[0]))

78 0.7087225914001465
新任 調查 局長 呂文忠 上任 ， 一席 將 全力 調查 危害 國家 安全 假新聞 的 談話 ， 讓 人民 感到 一陣 寒風 。 將國安 如此 無限 上綱 ， 怎 不 令人 憂心 調查局 走向 效忠 政黨 ， 而 非 效忠 國家 的 回頭路 ？ 調查 局源 於 國民 政府 時期 的 「 軍統 」 與 國民黨 「 中統 」 ， 政府 遷台 後 負責 保防 偵查 ， 在 戒 嚴時期 成為 政治 打手 而 惡名 昭彰 。 姑不論 其 時代 背景 ， 這一頁 「 東廠 」 局史絕 不 光彩 ， 也 逼使 調查 局展 開漫長 的 「 國家化 」 歷程 ， 致力 肅貪 、 掃毒 、 查賄 等 ， 才 重拾 人民 信賴 。 如今 調查局 官網 明白 揭示 兩個 維護國安 工作 重點 ： 對受 中共 派遣 或 運用 ， 在 台灣 從 事 涉密 情 蒐 、 發展 組織 或 進行 破壞 的 個 人 或 組織 ， 以及 境內 外籍人士 或團體 蒐集 政經 、 國防 、 高科技 機密 資料 等 行為 ， 進行 監控 與 偵處 。 規範 如此 詳細 ， 實著眼 於 公權力 何其 強大 ， 自須 有 具體 違 法事 證作 為 調查 前提 ， 不容 單憑 「 國家 安全 」 的 不 確定 法律 概念 入人 於 罪 ； 並且 ， 所謂 危害 國安假 新聞 一旦 成為執 政者 遮掩 失政 的 卸責話術 ， 對國家 危害 實遠 甚假 新聞 。 二 ○ 一二 總統 大選前 ， 不過 是 一則 國安會疑 透過 調查局 對 蔡 英文 陣營 監控 的 傳聞 報導 ， 即 引發 其 強烈 批判 ； 如今 掌權後竟 想 假調 查局 之手 、 藉 國安之名 開啟 更廣 更深 的 干 預言論 自由 之門 ， 蔡 政府 心態 確實 可議 。 「 我們 是 國家 的 調查員 … 」 ， 展抱 山莊 激昂 的 「 調查員 進行曲 」 依舊 繞梁 ， 但展 抱 人 可 真 甘心 淪為 政治 打手 ， 讓 「 老牌 東廠 復 活 」 的 烏雲罩 頂 ？  0.7087225914001465 316
3293 0.47007647156715393
蔡 英文 總統 最近 的 一些 作法 ， 讓 人 感到 矛盾 。 她 一面 向 原住民 大談 和解 ， 一面 卻 在 社會 上 搞 鬥爭

In [14]:
df['d2v_Vec']='0'
#infered_vectors_list = []
for i in range(0,3738):
    strl=df.iloc[i,0]
    test_text=strl.split(' ')
    #得到向量array命名為inferred_vector
    inferred_vector=model_dm.infer_vector(doc_words=test_text,alpha=0.025,steps=500)
    df.loc[i,'d2v_Vec']=list(inferred_vector)
    #infered_vectors_list.append(inferred_vector)
print(df.head())    

                                             content tag       title  \
0  民進 黨 的 主席 選戰 ， 儼 已成 為 「 保皇派 」 和 「 反 保皇派 」 之爭 。...   0       女皇的新衣   
1  葉俊榮 在 宣布 教育部 將發台 大校 長 聘書給 管中 閔 之前 ， 據 說 曾 致電 立...   0  葉部長「勇敢」的代價   
2  拗 了 又 拗 ， 教育部 長 葉俊榮終 於 宣布 將發台 大校 長 聘書給 管中 閔 ， ...   0    三百天才找回理性   
3  蘇貞昌將 接閣 揆 已 經傳 了 一陣子 。 儘 管 其 幕僚 否認 此 說 ， 但 多位 ...   0    聽說蘇貞昌接閣揆   
4  柯文 哲雙 城論壇 再 提 「 兩岸 一家 親 」 ， 又 吹 皺政壇 春水 。 對照 他 ...   0    柯文哲的如意算盤   

                                             d2v_Vec  
0  [-1.206782, -1.3442309, -5.4025793, -1.1229587...  
1  [-0.4745511, -5.0516253, -3.6623557, 0.4304008...  
2  [-3.8533144, -3.2074935, -0.8450376, 0.3748564...  
3  [-2.204379, -1.9464985, -1.8394303, 2.3343253,...  
4  [-1.9381931, 1.8546289, -2.243623, 0.55695444,...  


In [15]:
dfn = df.drop(["content"], axis=1)
dfn = dfn.drop(["title"], axis=1)

In [57]:
#dfn['d2v_Vec']

In [58]:
#dfn.to_csv("doc2vec.csv", encoding="utf-8", index=False)


In [39]:
from sklearn.model_selection import train_test_split
# axis = 1 drop行 axis = 0 drop列
# (題目90%, 題目10%, 答案90%, 答案10%)
dfn_train, dfn_test, tag_train, tag_test = train_test_split(dfn, 
                                   dfn["tag"], test_size=0.1)

In [40]:
#分類器
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier(n_neighbors=8)
clf=clf.fit(list(dfn_train['d2v_Vec']),tag_train)


In [125]:
#開始預測
from sklearn.metrics import accuracy_score
predict=clf.predict(list(dfn_test['d2v_Vec']))
#print("預測 :", predict)
#print("正確標籤 :", tag_test)
print("正確率 :",accuracy_score(tag_test,predict)*100,"%")

正確率 : 73.52941176470588 %


In [42]:
#使用混淆矩陣，確認一下分類錯誤
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(tag_test,predict)
pd.DataFrame(cm)

,0,1
0,100,92
1,7,175


In [114]:
#保存KNN模型
#import pickle 
with open('ModelSave/KNN_Label_P.pickle','wb')as f:
    pickle.dump(clf,f)

In [118]:
#隨便找一筆個資料來預測

# #綠
#strl="前總統李登輝因跌傷送台北榮總，雖然病情穩定許多，但還在復健養病，這次跨年只能在醫院度過。李登輝辦公室主任王燕軍表示，李前總統狀況很好，慢慢可以吃較為軟爛的食物，也能自己起床坐臥，感謝大家關心。李登輝何時可以出院？王燕軍表示，還沒討論到這一塊。不過北榮日前表示，醫療團隊會朝春節前出院的方向努力，讓李前總統回家過年。高齡95歲的李登輝去年11月29從起居室起身要到臥室時，不慎往前跌倒，當下撞到右側眉骨上方，除了鼻腔內側出血，也有些微腦出血。經北榮悉心治療，李的病情獲得控制，12月12日轉入普通病房，展開肌力與吞嚥復健"
#strl="桃園市長鄭文燦今出席中壢青埔的元旦升旗典禮許下5個願望，盼藉此讓桃園城市轉型，讓市民生活更好。新的一年，他同時期望民進黨可以檢討、改革、接地氣，以人民的感受為出發點，找出新的方向，一個政黨的生命在於人民的支持，人民的感受是最重要的，希望未來的團隊，能夠以此當作出發點。鄭文燦許下元旦的5個願望，包括航空城計畫、亞洲矽谷計畫、軌道建設、城市轉型基礎建設，以及照顧年輕人的政策。透過這些目標「轉大人」，讓桃園成為台灣的亮點城市，也成為台商回台首選，希望桃園繼續往前衝衝衝。對於媒體詢問，市長是否有個人的願望時，鄭文燦則說，當市長的每一天都在努力完成市民的願望，自己最大的願望，就是每一天的工作都要做好。他也祈願中華民國台灣，能夠往前走，每一步都能穩健也走出希望。"

#藍
strl="民進黨敗選後，蔡政府的演出越來越像沒定稿的「預告片」，不時在修改劇情，連導演也不曉得要如何收尾。最糟糕的是，「演員表」都排不定，不知誰要擔綱演出。例如，閣揆賴清德第一時間辭職，第一時間就被慰留；他隨即預告「堅定走人」，但蔡總統又宣稱「堅定留人」。這些，都只是「預告」。這些欲拒還迎的過程，本應在電影開拍之前全部敲定，現在，卻把幕後的拉拉扯扯都露給觀眾看。沒定稿的「預告片」，是指情節仍未構思完整，導演就拿出一些語焉不詳的片段來唬人，預告是為了掩飾茫然。在正常的民主國家，閣揆敗選就下台負責，新閣布局成形就上台開工。但在台灣，蔡政府卻在兩者之間游移，要下的下不去，該上的上不來，跟吳音寧和管中閔爭議時一樣。請問，台灣連一個正常民主國家的規格都跟不上了嗎？內閣人事喬不定，已經讓人看了心慌慌，但政府決策的出爾反爾，比未定稿的「預告片」更令人瘋狂。規劃已久的機車加裝ＡＢＳ系統政策，一推出，就遭總統一口否決。為了防疫安全，那廂對非洲豬瘟祭出大棒，這廂卻叫停洗選蛋政策，顯示任何政策都無定論。更驚奇的是「卡管」案，開拍時是綠色恐怖片，中途變成警匪教育片，收場卻是懸疑推理劇。一個結局，神祕的峰迴路轉，任觀眾各自解讀。"
#strl="國民黨發言人、文傳會副主委洪孟楷今天上午透過臉書宣布將投入明年新北市第一選區（淡水、林口、泰山、八里、三芝、石門）立委選舉，並喊出「世代交替和經驗傳承是現在的共識」，對此也已表明參選該區立委的國民黨前立委吳育昇表示，他樂見洪孟楷表態競爭，但他認為「世代交替」這個字眼不好，「難道在立院比洪孟楷年紀還大的都要被世代交替嗎？」吳育昇表示，國民黨是民主政黨，本來就可以公開黨內競爭，但洪孟楷參選打出「世代交替」，他認為不好、不妥，吳育昇說，2018年九合一大選韓國瑜掀起韓流、侯友宜獲得壓倒性支持，原因都不在年紀，若比洪孟楷年紀大的就要世代交替，立院比洪年紀大的立委，像費鴻泰、曾銘宗等人，是否也要被交替？吳育昇說，立委作為中央層級的民意代表，要有成熟的問政技巧，耕耘專業領域、完整歷練，並有宏觀格局，才有代表性，而非只是考量年紀。他在新北市立委第一選區連任3屆立委，對地方最瞭解，現在逐一實現的淡海輕軌、淡江大橋都是他爭取的建設，在地方服務有口碑。吳育昇也說，3年前因大環境不利所以落選，但他對地方的關懷沒有停過，去年他積極輔選侯友宜，侯友宜當選新北市長後也鼓勵他再參選立委為地方服務，因此去年九合一大選後，他也已明確對外表態將捲土重來挑戰該區立委。除洪孟楷、吳育昇外，也傳出地方勸進剛在議員選舉中拿下選區最高票的議員蔡淑君，蔡淑君去年底投票獲3萬8000多票，但光是在林口就拿下約3萬票，基層實力驚人。對此蔡淑君表示，她謝謝大家的肯定和鼓勵，是否參選立委，她仍審慎評估中。國民黨新北市黨部主委李乾龍表示，新北立委第一選區有多人表達意願，將等黨中央擬出提名、初選相關辦法後，再進行協調。"
#strl="國民黨發言人洪孟楷今早宣布投入2020立委選舉，參選第10屆新北市第一選區（淡水、林口、泰山、八里、三芝、石門）立委，將有機會對戰有「淡水蔡依林」之稱的現任綠委呂孫綾。洪孟楷表示，看到中央執政的弊病不斷，現在的執政黨無心想人民，只在意如何打擊政敵，對付國民黨；《勞動基準法》亂修、年金改革違反信賴原則、前瞻計畫是撒錢自肥、兩岸關係冷凍，他所熱愛的台灣與中華民國，還有多少個4年可以浪費呢？他將用要用年輕的熱忱，打一場正向陽光選戰，要用過去10年累積的專業，監督揭弊、爭取預算，不做負面攻擊、跳脫政黨鬥爭。兼任文傳會義務職副主委的洪孟楷今正式表態參選2020立委，他說，過去馬政府8年執政，當時他與國民黨並未有過多接觸，然在2016年國民黨在野後，當眾人選擇遠離，他毅然決然擔任黨職，並且為黨辯護，捍衛，可以說國民黨越弱時、更加不離不棄，因為相信民主政治需要良好的監督制衡，才能夠給民眾更好的生活。洪孟楷說，在政論節目中他只論理、不硬拗，因為相信「真理越辯越明」，但過去國民黨有不對的地方也絕不護短；在地方服務上，更是不分黨派、不分顏色，只要是對地方好的事物，孟楷勇於付出，樂於奉獻，這是他永遠的承諾。洪孟楷說，在這次九合一選舉中看到了「討厭民進黨」的勝利，但不見得是每個人都對於國民黨放心，因此，如果接下來的選舉，國民黨還是沒有任何改變，用一樣的面孔來面對選民，這樣怎麼能夠回應民眾對於政黨的期待呢？洪孟楷說，今年36歲的他，有完整的中央、地方政府歷練，曾是前台北縣副縣長蔡家福秘書，嘉義市政府文化局局長，也是行政院前院長江宜樺辦公室專門委員，他有信心能夠為公眾爭取權利，創造地方人人發大財"

In [119]:
#隨便找一筆個資料來預測
strl=" ".join(jieba.cut(strl))
test_text=strl.split(' ')
inferred_vector=model_dm.infer_vector(doc_words=test_text,alpha=0.025,steps=500)
ddf = pd.DataFrame(columns=['tag', 'd2v_Vec'])
ddf.loc[0,'d2v_Vec']=list(inferred_vector)

In [120]:
predict=clf.predict(list(ddf['d2v_Vec']))
print("預測 :", predict)

預測 : ['0']


In [121]:
# #叫模型出來用
# import pickle
# with open('ModelSave/doc2vec_Label_P.pickle','rb')as f:
#     model_dm=pickle.load(f)
    